In [23]:
import pandas as pd
import numpy as np
from scipy.stats import gmean, describe

In [26]:
df = pd.DataFrame([5,4,-4,-5,0, 0, 7])
df

,0
0,5
1,4
2,-4
3,-5
4,0
5,0
6,7


In [37]:
gmean([5,4, 7])

5.1924941018511035

In [40]:
df = pd.read_csv('/home/coder/Documents/GitHub/IDH-index/data/raw/data_ppr_2021_raw.csv', low_memory=False)
df = df[['AGEP', 'SCH', 'SCHL']]

# calcualte the mean of years of schooling
edu_sch = df[df['AGEP'] > 25].copy()
edu_sch['scholing'] =  edu_sch['SCHL']
edu_sch.reset_index(inplace=True)
edu_sch['scholing'].replace({3:1, 4:2, 5:3, 6:4, 7:5, 8:6, 9:7, 10:8, 11:9, 
                        12:10, 13:11, 14:12, 15:13, 15:13, 16:13, 17:13, 
                        18:13.5, 19:14, 20:15, 21:17, 22:19, 23:19, 24:23}, inplace=True)
edu_sch['enroled'] = np.where(edu_sch['scholing'] > 1, 1, 0)
mean_sch = edu_sch['scholing'].mean()

# get coeficient of ajustment
edu_sch['no_zero_schooling'] = 1 + edu_sch['scholing']

# calculate the expected years of schooling
edu_exp = df[df['AGEP'] < 25].copy()
edu_exp['enrolled'] = edu_exp['SCH'].apply(lambda x: 1 if x > 1 else 0)
edu_age = edu_exp.groupby(['AGEP'])[['AGEP','enrolled']].count()
edu_age['enrolled'] = edu_exp.groupby(['AGEP'])['enrolled'].sum()
edu_age['enrollment_rate'] = edu_age['enrolled'] / edu_age['AGEP']
edu_age = edu_age.rename (columns = {'AGEP': 'count'})
edu_age = edu_age.reset_index()
edu_age.drop([0,1,2,3,4], inplace=True)
exp_sch = edu_age['enrollment_rate'].sum()

# calculate index
edu_value = (mean_sch/15 + exp_sch/18) / 2

In [45]:
np.mean(edu_sch['no_zero_schooling'])

14.39070866309704

In [46]:
gmean(edu_sch['no_zero_schooling'])

13.449334301402379

In [44]:
edu_sch['no_zero_schooling'].mean()

14.39070866309704

In [52]:
edu_sch

,index,AGEP,SCH,SCHL,scholing,enroled,no_zero_schooling
0,0,40,1.0,19.0,14.0,1,15.0
1,2,61,1.0,18.0,13.5,1,14.5
2,3,63,1.0,12.0,10.0,1,11.0
3,4,58,1.0,16.0,13.0,1,14.0
4,5,84,1.0,18.0,13.5,1,14.5
...,...,...,...,...,...,...,...
93749,124663,56,1.0,16.0,13.0,1,14.0
93750,124664,47,1.0,22.0,19.0,1,20.0
93751,124665,46,1.0,21.0,17.0,1,18.0
93752,124668,61,1.0,17.0,13.0,1,14.0


In [53]:
# save to csv
edu_sch.to_csv('/home/coder/Documents/GitHub/IDH-index/data/iterm/no_zero_schooling.csv', index=False)